In [1]:
import numpy as np
import pandas  as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from scipy import linalg
from math import sqrt

In [2]:
client1 =np.load("FedELM/client1.npz")
client2 =np.load("FedELM/client2.npz")
client3 =np.load("FedELM/client3.npz")
client2.files

['X_a2_sc', 'y_a2']

In [3]:
client1["X_a1_sc"].shape

(412, 8)

In [4]:
X_a1_t,X_a1_s,y_a1_t,y_a1_s  = client1["X_a1_sc"][:100],client1["X_a1_sc"][100:],client1["y_a1"][:100],client1["y_a1"][100:]
X_a2_t,X_a2_s,y_a2_t,y_a2_s  = client2["X_a2_sc"][:1000],client2["X_a2_sc"][1000:],client2["y_a2"][:1000],client2["y_a2"][1000:]
X_a3_t, X_a3_s, y_a3_t, y_a3_s = client3["X_a3_sc"][:10000],client3["X_a3_sc"][10000:],client3["y_a3"][:10000],client3["y_a3"][10000:]
                                                  
X_a1_t.shape, y_a1_s.shape,X_a2_t.shape,y_a2_s.shape,X_a3_t.shape, X_a3_s.shape

((100, 8), (312,), (1000, 8), (4080,), (10000, 8), (5148, 8))

In [5]:
len(X_a1_t)

100

In [6]:
c1_par_L2= np.load("FedELM/client1_par_L2.npz") #neurons and alpha
c2_par_L2= np.load("FedELM/client2_par_L2.npz") #neurons and alpha
c3_par_L2= np.load("FedELM/client3_par_L2.npz") #neurons and alpha

# Client 1,2,3

In [14]:
class clientz1:
    l=20 # hidden neurones
    cl1 =np.load("FedELM/client1.npz")
    
    X_a1_t,X_a1_s,y_a1_t,y_a1_s  = cl1["X_a1_sc"][:10000],cl1["X_a1_sc"][10000:],cl1["y_a1"][:10000],cl1["y_a1"][10000:]
    input_size= len(X_a1_t[1])
    def __init__(self,batch_size, alpha=0.001):
        self.input_size = clientz1.input_size
        self.w = None
        self.b = None
        self.batch_size=batch_size
        self.reg_term = np.eye(clientz1.l) * alpha
          
   
    def get_Nfs(self):
        return self.input_size, self.input_size
    
    def set_W_b(self,w,b):
        self.w=w
        self.b=b
      
    
    def batches(self):
        
        batch_X=[]
        batch_y=[]
        
        if self.batch_size >= self.input_size:
            batch_X.append(X_a1_t)
            batch_y.append(y_a1_t)
             
        else:
            for i in range (0, self.input_size , self.batch_size):
                batch_X.append(X_a1_t[i:i + self.batch_size])
                batch_y.append(y_a1_t[i:i + self.batch_size])
            
        for X,y in zip(batch_X,batch_y): 
            
             # Compute the hidden layer output 
            H = np.tanh(np.dot(X, self.w) + self.b)  # Apply activation function (tanh)

            HTH = H.T @ H
            HTH_pinv= linalg.pinv(HTH) + self.reg_term
            HTy = H.T @ y
            
            #print(HTH)
            print(HTy)
            
            yield HTH_pinv, HTy               
                      
    def get_batch_weights(self):
        bac=self.batches()
        HTH_c1=[]
        HTy_c1=[]
        B= [] #np.zeros(self.input_size, clientz1.l)
        for HTHb,HTyb in bac:
            HTH_c1=HTHb
            HTy_c1=HTyb
        B=np.dot(linalg.pinv(HTH_c1),HTy_c1)

        average_matrix1 = np.mean(B, axis=0)
        print("average:", average_matrix1)
        print("B:", B)
        
        
        return B, average_matrix1 #HTH_c3, HTy_c3
    
    def predict_test(self, B):
                                    
        # Calculate the predicted output on the testing set
        H_test = np.tanh(np.dot(X_a1_s, self.w) + self.b)
        pred_test = np.dot(H_test, B)
        r2 = r2_score(y_a1_s, pred_test)
        return r2


In [15]:
class clientz2:
    
    l=20 # hidden neurones
    cl2 =np.load("FedELM/client2.npz")
    
    X_a2_t,X_a2_s,y_a2_t,y_a2_s  = cl2["X_a2_sc"][:10000],cl2["X_a2_sc"][10000:],cl2["y_a2"][:10000],cl2["y_a2"][10000:]
    input_size= len(X_a2_t[1])
    def __init__(self, batch_size, alpha=0.001):
        self.input_size = clientz2.input_size
        self.w = None
        self.b = None
        self.batch_size= batch_size
        self.reg_term = np.eye(clientz2.l) * alpha
          

    def get_Nfs(self):
        return self.input_size, self.input_size
    
    def set_W_b(self,w,b):
        self.w=w
        self.b=b
        
 
    def batches(self):
        
        batch_X=[]
        batch_y=[]
        
        
        if self.batch_size >= self.input_size:
            batch_X.append(X_a2_t)
            batch_y.append(y_a2_t)
             
        else:
            for i in range (0, self.input_size , self.batch_size):
                batch_X.append(X_a2_t[i:i + self.batch_size])
                batch_y.append(y_a2_t[i:i + self.batch_size])
            
            
        for X,y in zip(batch_X,batch_y): 
            
             # Compute the hidden layer output 
            H = np.tanh(np.dot(X, self.w) + self.b)  # Apply activation function (tanh)
    
            # Calculate the output weights using the Moore-Penrose pseudoinverse
            # B = np.dot(linalg.pinv(H.T.dot(H) + alpha *np. eye(l)), H.T.dot(y_train)) #output_weights
    
            HTH = H.T @ H
            HTH_pinv= linalg.pinv(HTH) + self.reg_term
            HTy = H.T @ y
            #print(HTH)
            #print(HTy)
            yield HTH_pinv, HTy
                      
                
    
    def get_batch_weights(self):
        bac=self.batches()
        HTH_c2=[]
        HTy_c2=[]
        B=[]
        for HTHb,HTyb in bac:
            HTH_c2=HTHb
            HTy_c2=HTyb
        B=np.dot(linalg.pinv(HTH_c2),HTy_c2)

        average_matrix2 = np.mean(B, axis=0)
        print("B:", B)
        #print("HTH_pinv:", HTH_pinv_c1)
        
            
        return B, average_matrix2 #HTH_c3, HTy_c3
    
    def predict_test(self, B):
                                    
        # Calculate the predicted output on the testing set
        H_test = np.tanh(np.dot(X_a2_s, self.w) + self.b)
        pred_test = np.dot(H_test, B)
        r2 = r2_score(y_a2_s, pred_test)
        return r2


In [16]:
class clientz3:
    l=20
    cl3 =np.load("FedELM/client3.npz")
    X_a3_t,X_a3_s,y_a3_t,y_a3_s  = cl3["X_a3_sc"][:10000],cl3["X_a3_sc"][10000:],cl3["y_a3"][:10000],cl3["y_a3"][10000:]
    input_size= len(X_a3_t[1])
    def __init__(self, batch_size, alpha=0.001):
        self.input_size = clientz3.input_size
        self.w = None
        self.b = None
        self.batch_size=batch_size
        self.reg_term = np.eye(clientz3.l) * alpha
           
    
    def get_Nfs(self):
        return self.input_size, self.input_size
    
    def set_W_b(self,w,b):
        self.w=w
        self.b=b
      
    
    def batches(self):
        
        batch_X=[]
        batch_y=[]
        
        
        if self.batch_size >= self.input_size:
            batch_X.append(X_a3_t)
            batch_y.append(y_a3_t)
             
        else:
            for i in range (0, self.input_size , self.batch_size):
                batch_X.append(X_a3_t[i:i + self.batch_size])
                batch_y.append(y_a3_t[i:i + self.batch_size])
            
        for X,y in zip(batch_X,batch_y): 
            # Compute the hidden layer output 
            H = np.tanh(np.dot(X, self.w) + self.b)  # Apply activation function (tanh)

            HTH = H.T @ H
             #HTH_pinv= linalg.pinv(HTH) + self.reg_term
            HTy = H.T @ y

            yield HTH, HTy
                      
    def get_batch_weights(self):
        bac=self.batches()
        HTH_c3=[]
        HTy_c3=[]
        B= [] #np.zeros(self.input_size, int(clientz3.l)
        for HTHb,HTyb in bac:
            HTH_c3=HTHb
            HTy_c3=HTyb
        B=np.dot(linalg.pinv(HTH_c3),HTy_c3)

        average_matrix3 = np.mean(B, axis=0)
        print("B:", B)
        #print("HTH_pinv:", HTH_pinv_c1)
       
            
        return B, average_matrix3 #HTH_c3, HTy_c3
    
    def predict_test(self, B):
                                    
        # Calculate the predicted output on the testing set
        H_test = np.tanh(np.dot(X_a3_s, self.w) + self.b)
        pred_test = np.dot(H_test, B)
        r2 = r2_score(y_a3_s, pred_test)
        return r2
        


#  Server side

In [17]:
def average_matrices(matrix1, matrix2, matrix3):
    # Check if the matrices have the same shape
    if matrix1.shape == matrix2.shape == matrix3.shape:
        # Sum the matrices element-wise
        average_matrix = (matrix1 + matrix2 + matrix3) / 3
        return average_matrix
    else:
        print("Matrices have different shapes and cannot be averaged.")
    
# Output the average matrix


In [22]:
#  For Client1
hidden_size = 20
c1=clientz1(batch_size=100,alpha=0.001)
c1_sample, c1_nf= c1.get_Nfs()

c1_w_inital = np.random.rand(c1_nf, hidden_size)
c1_bias_inital = np.random.rand(hidden_size)

In [23]:
c1.set_W_b(c1_w_inital, c1_bias_inital)
B1, ave=c1.get_batch_weights()

[ 40.73224446  54.55554361  -1.23031285  60.50985711  49.21622034
  53.73392141  -7.43670056  29.54065589  36.28135407  41.54495383
  55.4729184  -11.16880088  21.86469907  10.72219773  60.18706898
  -8.02079868  45.4339485   63.19388143  62.20485533   8.00736375]
average: 16133.34796063813
B: [19008.92405291 21442.74026809  6424.50123521 21844.10963604
 19712.67777385 22221.93393681  4056.24995711 16843.13824267
 18262.68042405 20213.72407812 22343.42448492  3177.3129655
 14188.63903976 10880.14537772 22635.91047808  4607.45758581
 20396.28038911 21923.77079847 22454.15406943 10029.18441907]


In [24]:
c1.predict_test(B1)

-752996132815.4528

In [25]:
#  For Client2
hidden_size = 20
c2=clientz2(batch_size=1000,alpha=0.001)
c2_sample, c2_nf= c2.get_Nfs()

c2_w_inital = np.random.rand(c2_nf, hidden_size)
c2_bias_inital = np.random.rand(hidden_size)



In [26]:
c2.set_W_b(c2_w_inital, c2_bias_inital)
B2, ave2=c2.get_batch_weights()

B: [ -927537.36285745  1678044.74311429 -1751930.58568975 -1663290.15502051
 -1735254.06201718 -1686178.80500202  1671221.06272752   879784.80826537
  -810601.85574736 -1742936.74208014  -596248.23964385 -1604533.94579762
 -1720048.087225   -1622838.12050285  -845619.58870644   983964.60779594
  -818273.68579004  1649255.88373942  -522378.41602451 -1721217.50573472]


In [28]:
c2.predict_test(B2)

-495019560179845.6

In [29]:
# For Client3
hidden_size = 20
c3=clientz3(batch_size=1000,alpha=0.001)
c3_sample, c3_nf= c3.get_Nfs()

c3_w_inital = np.random.rand(c3_nf, hidden_size)
c3_bias_inital = np.random.rand(hidden_size)


In [30]:
c3.set_W_b(c3_w_inital, c3_bias_inital)
B3, ave3=c3.get_batch_weights()

B3

B: [ 0.46449014  0.47361592 -1.77812215  0.4779571   0.56813994 -1.27859344
  0.11825827 -0.40824866  0.71636773 -0.46309237  1.15987426 -1.00163075
 -0.24687764 -0.12080871 -0.29985752  0.00901283  0.76016422 -1.13980996
  0.06895654  1.40429811]


array([ 0.46449014,  0.47361592, -1.77812215,  0.4779571 ,  0.56813994,
       -1.27859344,  0.11825827, -0.40824866,  0.71636773, -0.46309237,
        1.15987426, -1.00163075, -0.24687764, -0.12080871, -0.29985752,
        0.00901283,  0.76016422, -1.13980996,  0.06895654,  1.40429811])

In [31]:
c3.predict_test(B3)

0.1408493470830271

# Aggregate B

In [1002]:
average_matrices(B1,B2,B3)

array([-3.13612428e+01,  5.65583947e+05, -5.95819409e+05,  3.86999273e+05,
       -5.94150522e+05, -5.91280607e+05,  5.00308952e+05, -5.96011820e+05,
       -5.43786069e+05, -4.47821590e+05, -5.89625160e+05,  3.25578281e+05,
       -5.80974181e+05, -5.78295073e+05, -5.89634953e+05, -4.84358488e+05,
       -5.87855165e+05, -5.82835597e+05, -5.89534954e+05, -5.00779509e+05])